# About the dataset


In [1]:
import os
from urllib.request import urlretrieve
import numpy as np


url = "https://www.cs.cmu.edu/%7Espok/grimmtmp/"
dir_name = "data"


def download_data(url, filename, download_dir):
    """Download a file if not present, and make sure it's the right sizze"""

    os.makedirs(download_dir, exist_ok=True)

    if not os.path.exists(os.path.join(download_dir, filename)):
        filepath, _ = urlretrieve(url + filename, os.path.join(download_dir, filename))
    else:
        filepath = os.path.join(download_dir, filename)

    return filepath


# Number of files and their names to download
num_files = 209
filenames = [format(i, "03d") + ".txt" for i in range(1, num_files + 1)]

for fn in filenames:
    download_data(url, fn, dir_name)

for i in range(len(filenames)):
    file_exists = os.path.isfile(os.path.join(dir_name, filenames[i]))
    assert file_exists
print("{} files found.".format(len(filenames)))

209 files found.


In [2]:
import random

random_file = random.choice(os.listdir("./data"))
random_file

f = open(f"./data/{random_file}")
print(f.read())

Once upon a time lived a peasant and his wife, and the parson
of the village had a fancy for the wife, and had wished for
a long while to spend a whole day happily with her.  The
peasant woman, too, was quite willing.  One day, therefore, he
said to the woman, listen, my dear friend, I have now thought of
a way by which we can for once spend a whole day happily together.
I'll tell you what.  On wednesday, you must take to your bed, and
tell your husband you are ill, and as long as you complain and
act being ill properly, and go on doing so until sunday when I
have to preach, I will then say in my sermon that whosoever has
at home a sick child, a sick husband, a sick wife, a sick father,
a sick mother, a sick brother or whosoever else it may be,
and makes a pilgrimage to the gockerli hill in italy, where
you can get a peck of laurel-leaves for a kreuzer, the sick child,
the sick husband, the sick wife, the sick father, or sick mother,
the sick sister, or whosoever else it may be, will b

# Generating training, validation and test sets


In [3]:
from sklearn.model_selection import train_test_split

# Fix the random seed so we get the same output everytime
random_state = 54321

filenames = [os.path.join(dir_name, f) for f in os.listdir(dir_name)]

# First separate train and valid+test data
train_filenames, test_and_valid_filenames = train_test_split(
    filenames, test_size=0.2, random_state=random_state
)

# Separate valid+test data to validation and test data
valid_filenames, test_filenames = train_test_split(
    test_and_valid_filenames, test_size=0.5, random_state=random_state
)

for subset_id, subset in zip(
    ("train", "valid", "test"), (train_filenames, valid_filenames, test_filenames)
):
    print(
        "Got {} files in the {} dataset (e.g.{})".format(
            len(subset), subset_id, subset[:3]
        )
    )

Got 167 files in the train dataset (e.g.['data/199.txt', 'data/036.txt', 'data/043.txt'])
Got 21 files in the valid dataset (e.g.['data/088.txt', 'data/118.txt', 'data/016.txt'])
Got 21 files in the test dataset (e.g.['data/176.txt', 'data/195.txt', 'data/123.txt'])


# Analyze the vocabulary size.

I will be using bigrams to train the language model. The story will be split into units of two characters.
The characters will be converted to lowercase to reduce the input dimensionality. Using character-level bigrams helps to reduce the vocabulary, leading to faster model training


In [4]:
bigram_set = set()
# Go through each file in the training set
for fname in train_filenames:
    document = []  # This will hold all the text
    with open(fname, "r") as f:
        for row in f:
            # Convert text to lower case to reduce input dimensionality
            document.append(row.lower())
            # From the list of text we have, generate one long string
            # (containing all training stories)
        document = " ".join(document)
        # Update the set with all bigrams found
        bigram_set.update([document[i : i + 2] for i in range(0, len(document), 2)])
# Assign to a variable and print
n_vocab = len(bigram_set)
print("Found {} unique bigrams".format(n_vocab))

Found 696 unique bigrams


### Defining th etf.data pipeline

#### function generate_tf_dataset() that takes </br>

1. filenames - as list of filenames containing the text to be used for the model
2. ngram_width - widht of the n-grams to be extracted
3. window_size - lenght of the sequence of n-grams to be used to generate a single data point for the model
4. batch_size - Size of the batch
5. shuffle - (default to False ) whether to shuffle the data or not


In [5]:
import tensorflow as tf


def generate_tf_dataset(filenames, ngram_width, window_size, batch_size, shuffle=False):
    """Generate batched data from a list of files specified"""
    # Read the data found in the documents
    documents = []
    for f in filenames:
        doc = tf.io.read_file(f)
        doc = tf.strings.ngrams(  # Generate ngrams from the string
            tf.strings.bytes_split(
                # Create a list of chars from a string
                tf.strings.regex_replace(
                    # Replace new lines with space
                    tf.strings.lower(doc),  # Convert string to lower case
                    "\n",
                    " ",
                )
            ),
            ngram_width,
            separator="",
        )
    documents.append(doc.numpy().tolist())
    # documents is a list of list of strings, where each string is a story
    # From that we generate a ragged tensor
    documents = tf.ragged.constant(documents)

    # Create a dataset where each row in the ragged tensor would be a sample
    doc_dataset = tf.data.Dataset.from_tensor_slices(documents)

    # perform a quick transformation -tf.strings.ngrams
    # would generate all the ngrams (e.g. abcd => ab, bc, cd) with
    # overlaop, however for our data we do not need the overlap, so we need to skip the overlapping ngrams

    doc_dataset = doc_dataset.map(lambda x: x[::ngram_width])

    # using window function to generate windows from text
    # for a text sequence with window_size 3 and shift 1, you get e.g ab, cd, ef, gh, ij,... =>  [ab, cd, ef], [cd, ef, gh], [ef, gh, ij],...
    # each of these windows is a single training sequence for the model

    doc_dataset = doc_dataset.flat_map(
        lambda x: tf.data.Dataset.from_tensor_slices(x)
        .window(size=window_size + 1, shift=int(window_size * 0.75))
        .flat_map(lambda window: window.batch(window_size + 1, drop_remainder=True))
    )

    # From each windowed sequence we generate input and target tuple
    # e.g. [ab, cd, ef] -> ([ab, cd], [cd, ef])
    doc_dataset = doc_dataset.map(lambda x: (x[:-1], x[1:]))

    # Batch the data
    doc_dataset = doc_dataset.batch(batch_size=batch_size)

    # Shuffle the data if required
    doc_dataset = (
        doc_dataset.shuffle(buffer_size=batch_size * 10) if shuffle else doc_dataset
    )

    # return data
    return doc_dataset

2024-10-12 13:25:18.207333: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-10-12 13:25:18.216162: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:485] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-10-12 13:25:18.225917: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:8454] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-10-12 13:25:18.228694: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1452] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-10-12 13:25:18.236283: I tensorflow/core/platform/cpu_feature_guar

# Hyperparameters


In [6]:
ngram_length = 2
batch_size = 256
window_size = 128

train_ds = generate_tf_dataset(
    train_filenames, ngram_length, window_size, batch_size, shuffle=True
)
valid_ds = generate_tf_dataset(valid_filenames, ngram_length, window_size, batch_size)

test_ds = generate_tf_dataset(test_filenames, ngram_length, window_size, batch_size)

I0000 00:00:1728753919.342110  179994 cuda_executor.cc:1001] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
I0000 00:00:1728753919.360726  179994 cuda_executor.cc:1001] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
I0000 00:00:1728753919.360769  179994 cuda_executor.cc:1001] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
I0000 00:00:1728753919.364126  179994 cuda_executor.cc:1001] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
I0000 00:00:1728753919.364164  179994 cuda_executor.cc:1001] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
I0000 00:0

In [7]:
# looking at some data generated by the function

ds = generate_tf_dataset(train_filenames, 2, window_size=10, batch_size=1).take(5)

for record in ds:
    print(record[0].numpy(), "=>", record[1].numpy())

[[b'th' b'er' b'e ' b'wa' b's ' b'on' b'ce' b' a' b'n ' b'en']] => [[b'er' b'e ' b'wa' b's ' b'on' b'ce' b' a' b'n ' b'en' b'ch']]
[[b' a' b'n ' b'en' b'ch' b'an' b'tr' b'es' b's,' b' w' b'ho']] => [[b'n ' b'en' b'ch' b'an' b'tr' b'es' b's,' b' w' b'ho' b' h']]
[[b's,' b' w' b'ho' b' h' b'ad' b' t' b'hr' b'ee' b' s' b'on']] => [[b' w' b'ho' b' h' b'ad' b' t' b'hr' b'ee' b' s' b'on' b's ']]
[[b'ee' b' s' b'on' b's ' b'wh' b'o ' b'lo' b've' b'd ' b'ea']] => [[b' s' b'on' b's ' b'wh' b'o ' b'lo' b've' b'd ' b'ea' b'ch']]
[[b've' b'd ' b'ea' b'ch' b' o' b'th' b'er' b' a' b's ' b'br']] => [[b'd ' b'ea' b'ch' b' o' b'th' b'er' b' a' b's ' b'br' b'ot']]


2024-10-12 13:25:25.267236: I tensorflow/core/framework/local_rendezvous.cc:404] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence


# Implementing the language model


### Defining the TExtVectorization layer


In [8]:
# import tensorflow.keras.layers as layers
import tensorflow.keras.models as models
from keras import layers

# The vectorization layer that will convert string bigrams to IDs
text_vectorizer = layers.TextVectorization(
    max_tokens=n_vocab,
    standardize=None,
    split=None,
    input_shape=(window_size,),
    #     encoding="utf-8",
    #     output_mode="int",
)

/home/mbeleck/anaconda3/envs/tf2-cuda/lib/python3.12/site-packages/keras/src/layers/preprocessing/text_vectorization.py:340: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(name=name, **kwargs)


In [9]:
text_vectorizer.adapt(train_ds)
text_vectorizer.get_vocabulary()[:10]

2024-10-12 13:25:25.344884: I tensorflow/core/framework/local_rendezvous.cc:404] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence


['', '[UNK]', 'e ', 'th', 'he', ' t', 't ', ' w', ' a', 'd ']

In [10]:
train_ds = train_ds.map(lambda x, y: (x, text_vectorizer(y)))
valid_ds = valid_ds.map(lambda x, y: (x, text_vectorizer(y)))


# train_ds = train_ds.map(lambda x, y: (text_vectorizer(x), text_vectorizer(y)))
# valid_ds = valid_ds.map(lambda x, y: (text_vectorizer(x), text_vectorizer(y)))
# test_ds = test_ds.map(lambda x, y: (text_vectorizer(x), text_vectorizer(y)))

# Defining the LSTM model. The model will have

1. The previously trained TextVectorization layer
2. An embedding layer randomly initialized and jointly trained with the model
3. Two LSTM layers each with 512 and 256 nodes respectively
4. A fully-connected hidden lyaer with 1024 nodes and ReLU activation function
5. The final prediction layer with n_vocab nodes and softmax activation


In [11]:
import tensorflow.keras.backend as K
from tensorflow.keras import Input

K.clear_session()
lm_model = tf.keras.Sequential(
    [
        Input(shape=(10,), dtype=tf.string),
        text_vectorizer,
        layers.Embedding(n_vocab + 2, 96),
        layers.LSTM(512, return_state=False, return_sequences=True),
        layers.LSTM(256, return_state=False, return_sequences=True),
        layers.Dense(1024, activation="relu"),
        layers.Dropout(0.5),
        layers.Dense(n_vocab, activation="softmax"),
    ]
)

lm_model.summary()

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ text_vectorization              │ (None, None)           │             0 │
│ (TextVectorization)             │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ embedding (Embedding)           │ (None, None, 96)       │        67,008 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lstm (LSTM)                     │ (None, None, 512)      │     1,247,232 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lstm_1 (LSTM)                   │ (None, None, 256)      │       787,456 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, None, 1024)     │       263,168 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ (None, None, 1024)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, None, 696)      │       713,400 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 3,078,264 (11.74 MB)

 Trainable params: 3,078,264 (11.74 MB)

 Non-trainable params: 0 (0.00 B)

# Defining metrics and compiling the model


In [12]:
# using Perplexity


class PerplexityMetric(tf.keras.metrics.Mean):
    def __init__(self, name="perplexity", **kwargs):
        super().__init__(name=name, **kwargs)
        self.cross_entropy = tf.keras.losses.SparseCategoricalCrossentropy(
            from_logits=False, reduction="none"
        )

    def _calculate_perplexity(self, real, pred):
        # The next 4 lines zero-out the padding from loss
        # calculations, this follows the logic from:
        # https://www.tensorflow.org/beta/tutorials/text/transformer#loss_
        # and_metrics
        loss_ = self.cross_entropy(real, pred)
        # Calculating the perplexity steps:
        step1 = K.mean(loss_, axis=-1)
        perplexity = K.exp(step1)
        return perplexity

    def update_state(self, y_true, y_pred, sample_weight=None):
        perplexity = self._calculate_perplexity(y_true, y_pred)
        super().update_state(perplexity)

### Compile the model using

1. Sparse categorical cross-entropy as loss function
2. Adam as the optimizer
3. Accuracy and perplexity as metrics


In [13]:
lm_model.compile(
    loss="sparse_categorical_crossentropy",
    optimizer="adam",
    metrics=["accuracy", PerplexityMetric()],
)

for i, label in train_ds.take(1):
    print(label)

tf.Tensor(
[[ 18   2  59 ... 183  45 127]
 [ 57 128  71 ...  52  90 115]
 [250   8  21 ...  10  49   8]
 ...
 [109  41  88 ... 151 193   7]
 [ 58   3   5 ...  42  68  78]
 [  8  30   5 ...  32  37 148]], shape=(28, 128), dtype=int64)


# Training the Model


In [14]:
lm_model.fit(train_ds, validation_data=valid_ds, epochs=600)

Epoch 1/600


2024-10-12 13:25:27.535094: I external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:531] Loaded cuDNN version 8907


      1/Unknown 2s 2s/step - accuracy: 0.0011 - loss: 6.5455 - perplexity: 696.1003

2024-10-12 13:25:28.095790: I tensorflow/core/framework/local_rendezvous.cc:404] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence
	 [[{{node IteratorGetNext}}]]
2024-10-12 13:25:28.095827: I tensorflow/core/framework/local_rendezvous.cc:423] Local rendezvous recv item cancelled. Key hash: 14469679214315600240
2024-10-12 13:25:28.095861: I tensorflow/core/framework/local_rendezvous.cc:423] Local rendezvous recv item cancelled. Key hash: 14590964112509797676
2024-10-12 13:25:28.095941: I tensorflow/core/framework/local_rendezvous.cc:423] Local rendezvous recv item cancelled. Key hash: 13199244583136268423
2024-10-12 13:25:28.095973: I tensorflow/core/framework/local_rendezvous.cc:423] Local rendezvous recv item cancelled. Key hash: 4201179678154928553
2024-10-12 13:25:28.095976: I tensorflow/core/framework/local_rendezvous.cc:427] Local rendezvous send item cancelled. Key hash: 17283632123258503901
2024-10-12 13:25:28.095978: I tensorflow/core/framework/local_rende

1/1 ━━━━━━━━━━━━━━━━━━━━ 2s 2s/step - accuracy: 0.0011 - loss: 6.5455 - perplexity: 696.1003 - val_accuracy: 0.0337 - val_loss: 6.5414 - val_perplexity: 693.2593
Epoch 2/600
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 108ms/step - accuracy: 0.0299 - loss: 6.5410 - perplexity: 692.9920 - val_accuracy: 0.0325 - val_loss: 6.5343 - val_perplexity: 688.3436
Epoch 3/600
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 112ms/step - accuracy: 0.0335 - loss: 6.5334 - perplexity: 687.7166 - val_accuracy: 0.0295 - val_loss: 6.5133 - val_perplexity: 674.0640
Epoch 4/600


...

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 138ms/step - accuracy: 0.9576 - loss: 0.2043 - perplexity: 1.2275 - val_accuracy: 0.1238 - val_loss: 13.0592 - val_perplexity: 855697.9375
Epoch 597/600
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 132ms/step - accuracy: 0.9556 - loss: 0.2009 - perplexity: 1.2236 - val_accuracy: 0.1216 - val_loss: 13.0819 - val_perplexity: 868378.6250
Epoch 598/600
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 126ms/step - accuracy: 0.9581 - loss: 0.2011 - perplexity: 1.2237 - val_accuracy: 0.1233 - val_loss: 13.1151 - val_perplexity: 910044.2500
Epoch 599/600


2024-10-12 13:26:56.765310: I tensorflow/core/framework/local_rendezvous.cc:423] Local rendezvous recv item cancelled. Key hash: 14469679214315600240
2024-10-12 13:26:56.765352: I tensorflow/core/framework/local_rendezvous.cc:423] Local rendezvous recv item cancelled. Key hash: 14590964112509797676
2024-10-12 13:26:56.765360: I tensorflow/core/framework/local_rendezvous.cc:423] Local rendezvous recv item cancelled. Key hash: 13199244583136268423
2024-10-12 13:26:56.765378: I tensorflow/core/framework/local_rendezvous.cc:423] Local rendezvous recv item cancelled. Key hash: 4201179678154928553
2024-10-12 13:26:56.765381: I tensorflow/core/framework/local_rendezvous.cc:427] Local rendezvous send item cancelled. Key hash: 17283632123258503901
2024-10-12 13:26:56.765383: I tensorflow/core/framework/local_rendezvous.cc:427] Local rendezvous send item cancelled. Key hash: 799685734784530081
2024-10-12 13:26:56.765385: I tensorflow/core/framework/local_rendezvous.cc:427] Local rendezvous send 

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 132ms/step - accuracy: 0.9556 - loss: 0.2013 - perplexity: 1.2236 - val_accuracy: 0.1221 - val_loss: 13.1260 - val_perplexity: 915227.8125
Epoch 600/600
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 82ms/step - accuracy: 0.9576 - loss: 0.1969 - perplexity: 1.2186

2024-10-12 13:26:57.035384: I tensorflow/core/framework/local_rendezvous.cc:423] Local rendezvous recv item cancelled. Key hash: 13199244583136268423
2024-10-12 13:26:57.035421: I tensorflow/core/framework/local_rendezvous.cc:423] Local rendezvous recv item cancelled. Key hash: 4201179678154928553
2024-10-12 13:26:57.035427: I tensorflow/core/framework/local_rendezvous.cc:423] Local rendezvous recv item cancelled. Key hash: 16234665221074822072
2024-10-12 13:26:57.035430: I tensorflow/core/framework/local_rendezvous.cc:423] Local rendezvous recv item cancelled. Key hash: 14590964112509797676
2024-10-12 13:26:57.082209: I tensorflow/core/framework/local_rendezvous.cc:423] Local rendezvous recv item cancelled. Key hash: 7956575660870928627
2024-10-12 13:26:57.082263: I tensorflow/core/framework/local_rendezvous.cc:423] Local rendezvous recv item cancelled. Key hash: 7454989135351635087
2024-10-12 13:26:57.082273: I tensorflow/core/framework/local_rendezvous.cc:423] Local rendezvous recv 

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 140ms/step - accuracy: 0.9576 - loss: 0.1969 - perplexity: 1.2186 - val_accuracy: 0.1206 - val_loss: 13.1171 - val_perplexity: 899146.5000


In [15]:
test_ds = test_ds.map(lambda x, y: (x, text_vectorizer(y)))


lm_model.evaluate(test_ds)

      1/Unknown 0s 46ms/step - accuracy: 0.1198 - loss: 12.5410 - perplexity: 636978.4375

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 58ms/step - accuracy: 0.1198 - loss: 12.5410 - perplexity: 636978.4375


2024-10-12 13:26:57.318611: I tensorflow/core/framework/local_rendezvous.cc:423] Local rendezvous recv item cancelled. Key hash: 7956575660870928627
2024-10-12 13:26:57.318668: I tensorflow/core/framework/local_rendezvous.cc:423] Local rendezvous recv item cancelled. Key hash: 7454989135351635087
2024-10-12 13:26:57.318681: I tensorflow/core/framework/local_rendezvous.cc:423] Local rendezvous recv item cancelled. Key hash: 16794058650538835827
2024-10-12 13:26:57.318690: I tensorflow/core/framework/local_rendezvous.cc:423] Local rendezvous recv item cancelled. Key hash: 9096993371179739650


[12.541015625, 0.1197916641831398, 636978.4375]

In [16]:
# Define inputs to the model
inp = tf.keras.layers.Input(dtype=tf.string, shape=(1,))
inp_state_c_lstm = tf.keras.layers.Input(shape=(512,))
inp_state_h_lstm = tf.keras.layers.Input(shape=(512,))
inp_state_c_lstm_1 = tf.keras.layers.Input(shape=(256,))
inp_state_h_lstm_1 = tf.keras.layers.Input(shape=(256,))
text_vectorized_out = lm_model.get_layer("text_vectorization")(inp)


# Define embedding layer and output
emb_layer = lm_model.get_layer("embedding")
emb_out = emb_layer(text_vectorized_out)

# Defining a LSTM layers and output
lstm_layer = tf.keras.layers.LSTM(512, return_state=True, return_sequences=True)
lstm_out, lstm_state_c, lstm_state_h = lstm_layer(
    emb_out, initial_state=[inp_state_c_lstm, inp_state_h_lstm]
)
lstm_1_layer = tf.keras.layers.LSTM(
    256,
    return_state=True,
    return_sequences=True,
)
lstm_1_out, lstm_1_state_c, lstm_1_state_h = lstm_1_layer(
    lstm_out, initial_state=[inp_state_c_lstm_1, inp_state_h_lstm_1]
)


# Defining a Dense layer and output
dense_out = lm_model.get_layer("dense")(lstm_1_out)

# Defining the final Dense layer and output
final_out = lm_model.get_layer("dense_1")(dense_out)

# Copy the weights from the original model
lstm_layer.set_weights(lm_model.get_layer("lstm").get_weights())
lstm_1_layer.set_weights(lm_model.get_layer("lstm_1").get_weights())

# Define final model
infer_model = tf.keras.models.Model(
    inputs=[
        inp,
        inp_state_c_lstm,
        inp_state_h_lstm,
        inp_state_c_lstm_1,
        inp_state_h_lstm_1,
    ],
    outputs=[final_out, lstm_state_c, lstm_state_h, lstm_1_state_c, lstm_1_state_h],
)

In [17]:
### Generating new text with the model

In [54]:
import numpy as np

text = [
    "When adam and eve were driven out of paradise, they werecompelled to build a house for themselves on barren ground"
]
seq = [text[0][i : i + 2] for i in range(0, len(text[0]), 2)]
print("Making predictions from a {} element long input".format(len(seq)))

for c in seq:
    print(np.array([[c]]))

Making predictions from a 12 element long input
[['Wh']]
[['en']]
[[' S']]
[['yl']]
[['vi']]
[['a ']]
[['wa']]
[['s ']]
[['a ']]
[['ch']]
[['il']]
[['d,']]


In [58]:
import numpy as np

vocabulary = infer_model.get_layer("text_vectorization").get_vocabulary()
index_word = dict(zip(range(len(vocabulary)), vocabulary))
# Reset the state of the model initially

# infer_model.reset_states()

# Defining the initial state as all zeros
state_c = np.zeros(shape=(1, 512))
state_h = np.zeros(shape=(1, 512))
state_c_1 = np.zeros(shape=(1, 256))
state_h_1 = np.zeros(shape=(1, 256))


# Recursively update the model by assigning new state to state
for c in seq:
    # print(c)
    out, state_c, state_h, state_c_1, state_h_1 = infer_model.predict(
        [
            tf.convert_to_tensor([[c]], dtype=tf.string),
            state_c,
            state_h,
            state_c_1,
            state_h_1,
        ]
    )

# Get final prediction after feeding the input string
wid = int(np.argmax(out[0], axis=-1).ravel())
word = index_word[wid]
text.append(word)
# Define first input to generate text recursively from
x = np.array([[word]])


for _ in range(200):
    # Get the next output and state
    x_tensor = tf.convert_to_tensor(x, dtype=tf.string)
    out, state_c, state_h, state_c_1, state_h_1 = infer_model.predict(
        [x_tensor, state_c, state_h, state_c_1, state_h_1]
    )
    # Get the word id and the word from out
    out_argsort = np.argsort(out[0], axis=-1).ravel()
    wid = int(out_argsort[-1])
    word = index_word[wid]

    # If the word ends with space, we introduce a bit of randomness
    # Essentially pick one of the top 3 outputs for that timestep
    # depending on their likelihood

    if word.endswith(" "):
        if np.random.normal() > 0.5:
            width = 5
            i = np.random.choice(
                list(range(-width, 0)),
                p=out_argsort[-width:] / out_argsort[-width:].sum(),
            )
            wid = int(out_argsort[i])
            word = index_word[wid]
    # Append the prediction
    text.append(word)
    # Recursively make the current prediction the next input
    x = np.array([[word]])
# Print the final output

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 18ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 19ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 19ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 19ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step


/tmp/ipykernel_179994/1016762860.py:30: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  wid = int(np.argmax(out[0], axis=-1).ravel())


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 18ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 19ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 18ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 18ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 18ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 19ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 20ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 18ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 19ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 19ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 18ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 18ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 18ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 18ms/step
1/1 ━━━━━━━━

In [56]:
print("\n")
print("=" * 60)
print("Final text: ")
# pred = "".join(text)

pred = []

for word in text:
    if "." not in word:
        pred.append(word)
    else:
        pred.append(word)
        # print('\n')
        print("".join(pred))
        pred = []



Final text: 
When Sylvia was a child,ve everything, sain the castle. 
he entered and went through all them to fi set free.
  those, however, who triets, anll of wrinkles, bleary eyes, and red hair.
g daver which bears in its body a red-hot egg, and in the egh you should encounter, third soand toem bore his humang to the spring, where third soe soanl resume mron.
 the youth. 
 you doness onruwearly and the words passed his lips than he was waiting to, and as we are equally strong, neither of us can get the better of the castlad seamseln the air.
  each of them bore his humase what i am like, loos the king's daughter.
  but how shocke eow what i am like, look in the mirror - it doek nor, whose beauty, it let the egg fall. 


In [ ]:
# The model is able to generate text, but most of it doesn't make sense. next step, improving the model